In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LSTM
import torch.optim as optim
from torch.autograd import Variable

In [2]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd

from datetime import date
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import warnings
warnings.simplefilter("ignore", UserWarning)

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

In [3]:
def episode_gen(data, seq_length,output_size):
    x = []
    y = []
    #Loop through data, adding input data to x array and output data to y array
    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length - output_size)]
        _y = data[i+seq_length - output_size:i + seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [4]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

df['signal'] = df['signal'].replace({'SignalNone': 0, 'SignalLong': 1, 'SignalShort': -1})

/var/folders/x1/twdts0414gn1_4rw0fsqsphm0000gn/T/ipykernel_1864/1816512338.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['signal'] = df['signal'].replace({'SignalNone': 0, 'SignalLong': 1, 'SignalShort': -1})


In [5]:
shrink_size = int(0.1*(len(df)))
mini_df = df[-shrink_size:].copy()

In [6]:
data = vbt.Data.from_data(mini_df)

In [7]:
open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

adx = vbt.ADX.run(high_price, low_price, close_price, window=14)
atr = vbt.ATR.run(high_price, low_price, close_price, window=14)
bbands = vbt.BBANDS.run(close_price, window=14)
rsi = vbt.RSI.run(close_price)
sma = vbt.MA.run(close_price, window=20)
strend = vbt.SUPERTREND.run(high_price, low_price, close_price, period=7, multiplier=3)

data.data['symbol']['ADX'] = adx.adx
data.data['symbol']['ATR'] = atr.atr
data.data['symbol']['BBAND'] = bbands.bandwidth
data.data['symbol']['RSI'] = rsi.rsi
data.data['symbol']['SMA'] = sma.ma
data.data['symbol']['STREND'] = strend.trend
data.data['symbol'] = data.data['symbol'].dropna()

In [8]:
predictor_list = ['Open', 'High', 'Low', 'Close', 'Volume', 'ADX', 'ATR', 'BBAND', 'RSI', 'SMA', 'STREND']

X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

In [9]:
test_size = int(0.3*(len(X)))
X_train = X[:-test_size]
X_test = X[-test_size:]

y_train = y[:-test_size]
y_test = y[-test_size:]

In [10]:
scl = StandardScaler()

X_train_scaled=scl.fit_transform(X_train)
X_train= X_train.assign(Open=X_train_scaled[:, 0])
X_train= X_train.assign(High=X_train_scaled[:, 1])
X_train= X_train.assign(Low=X_train_scaled[:, 2])
X_train= X_train.assign(Close=X_train_scaled[:, 3])
X_train= X_train.assign(Volume=X_train_scaled[:, 4])
X_train= X_train.assign(ADX=X_train_scaled[:, 5])
X_train= X_train.assign(ATR=X_train_scaled[:, 6])
X_train= X_train.assign(BBAND=X_train_scaled[:, 7])
X_train= X_train.assign(RSI=X_train_scaled[:, 8])
X_train= X_train.assign(SMA=X_train_scaled[:, 9])
X_train= X_train.assign(STREND=X_train_scaled[:, 10])


X_test_scaled=scl.transform(X_test)

X_test= X_test.assign(Open=X_test_scaled[:, 0])
X_test= X_test.assign(High=X_test_scaled[:, 1])
X_test= X_test.assign(Low=X_test_scaled[:, 2])
X_test= X_test.assign(Close=X_test_scaled[:, 3])
X_test= X_test.assign(Volume=X_test_scaled[:, 4])
X_test= X_test.assign(ADX=X_test_scaled[:, 5])
X_test= X_test.assign(ATR=X_test_scaled[:, 6])
X_test= X_test.assign(BBAND=X_test_scaled[:, 7])
X_test= X_test.assign(RSI=X_test_scaled[:, 8])
X_test= X_test.assign(SMA=X_test_scaled[:, 9])
X_test= X_test.assign(STREND=X_test_scaled[:, 10])

In [11]:
seq_length = 8
output_size = 3

In [12]:
# Convert X and y into numpy arrays
X_np = X_train.to_numpy(dtype=float)
y_np = y_train.to_numpy(dtype=float)

# Convert the numpy arrays into torch tensors
X_tensor = torch.tensor(X_np, dtype=torch.float)
y_tensor = torch.tensor(y_np, dtype=torch.float).unsqueeze(1)  # Adding an extra dimension to y to match input requirements


In [13]:
num_epochs = 2000
learning_rate = 0.03

In [14]:
lstm = LSTM(11,20, batch_first = True)
lin = nn.Linear(20,3)
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam([
            {'params': lstm.parameters()},
            {'params': lin.parameters()}
        ], lr=learning_rate)

In [15]:
# Train the model
for epoch in range(num_epochs):
    #run model
    outputs, (h_n, c_n) = lstm(X_tensor)
    out = lin(h_n)
    out = 0.5 + F.sigmoid(out)
    optimizer.zero_grad()

    loss = criterion(out, y_tensor) #calculate loss function

    loss.backward() #backprop

    optimizer.step() #gradient descent

    #Output loss functions every 500 epochs so we can make sure the model is training
    if epoch % 2 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

Epoch: 0, loss: 1.05626
Epoch: 2, loss: 0.65146
Epoch: 4, loss: 0.43511
Epoch: 6, loss: 0.36505
Epoch: 8, loss: 0.34025
Epoch: 10, loss: 0.32988
Epoch: 12, loss: 0.32502
Epoch: 14, loss: 0.32251
Epoch: 16, loss: 0.32110
Epoch: 18, loss: 0.32027
Epoch: 20, loss: 0.31974
Epoch: 22, loss: 0.31939
Epoch: 24, loss: 0.31915
Epoch: 26, loss: 0.31898
Epoch: 28, loss: 0.31886
Epoch: 30, loss: 0.31876
Epoch: 32, loss: 0.31869
Epoch: 34, loss: 0.31864
Epoch: 36, loss: 0.31859
Epoch: 38, loss: 0.31856
Epoch: 40, loss: 0.31853
Epoch: 42, loss: 0.31851
Epoch: 44, loss: 0.31849
Epoch: 46, loss: 0.31847
Epoch: 48, loss: 0.31846
Epoch: 50, loss: 0.31844
Epoch: 52, loss: 0.31843
Epoch: 54, loss: 0.31843
Epoch: 56, loss: 0.31842
Epoch: 58, loss: 0.31841
Epoch: 60, loss: 0.31840
Epoch: 62, loss: 0.31840
Epoch: 64, loss: 0.31839
Epoch: 66, loss: 0.31839
Epoch: 68, loss: 0.31838
Epoch: 70, loss: 0.31838
Epoch: 72, loss: 0.31837
Epoch: 74, loss: 0.31837
Epoch: 76, loss: 0.31837
Epoch: 78, loss: 0.31836
Epoch